**Imports**

In [ ]:
import os
import json
import zipfile
import boto3
from botocore.client import Config

**Retrieve MinIO credentials and initialize client**

In [ ]:
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = "models"
region = os.getenv('AWS_DEFAULT_REGION')

s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

**Unzip the replacement.zip file from previous pipeline node**

In [ ]:
with zipfile.ZipFile('replacement.zip', 'r') as zipf:
    zipf.extractall()

**Load from json if a new model should replace the current production model**

In [ ]:
with open('should_replace.json', 'r') as f:
    decision = json.load(f)

**Get the project name from an environment variable**

In [ ]:
project_name_param = os.getenv('project_name')

**Define function to remove current production model from production file in MinIO object storage**

In [ ]:
def empty_production_folder():
    objects_to_delete = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=f"{project_name_param}/production/")

    if 'Contents' in objects_to_delete:
        delete_keys = {'Objects': []}
        for obj in objects_to_delete['Contents']:
            delete_keys['Objects'].append({'Key': obj['Key']})
        s3_client.delete_objects(Bucket=bucket_name, Delete=delete_keys)
    else:
        print("No objects found")

**Define function to find the file name of the model**

In [ ]:
def find_model_file():
    for file in os.listdir(os.getcwd()):
        if file.endswith('.onnx'):
            return file

    return None

**Get the model file name and define function to upload the new model to the production folder in MinIO object storage**

In [ ]:
model = find_model_file()
folder_name = f"{project_name_param}/production/"

def upload_to_production_folder():
    s3_client.upload_file(model, bucket_name, folder_name + model)

**If model should be replaced, perform actions to replace current production model with the new one**

In [ ]:
if decision['replace']:
    empty_production_folder()
    upload_to_production_folder()
else:
    print("Current model retained in production.")